# Dfs2

**This material have been updated to use MIKE IO 1.0, which currently avilable in a [beta version](https://pypi.org/project/mikeio/1.0b2/)**

[DFS - Reference manual](https://docs.mikepoweredbydhi.com/core_libraries/dfs/dfs-file-system/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mikeio

## Reading data

In [ ]:
ds = mikeio.read("data/gebco_sound.dfs2")
ds

In [ ]:
type(ds)

In [ ]:
ds.shape

In [ ]:
elevation = ds['Elevation']
type(elevation)

In [ ]:
elevation.shape

In [ ]:
elevation.plot()

## Spatial axes

In [ ]:
ds.geometry

## Extract data at location

In [ ]:
stn = 12.74792, 55.865
da_stn = elevation.interp(x=stn[0], y=stn[1])
da_stn

In [ ]:
da_stn.values

In [ ]:
elevation.plot()
plt.scatter(stn[0],stn[1],marker='*',s=500,c='black')
plt.text(12.85, 56.0, "Sweden")
plt.text(12.3, 55.8, "Denmark")

In [ ]:
import numpy as np

# Find minimum value
idx_1d = np.nanargmin(elevation.values)
idx_1d

In [ ]:
idx = np.unravel_index(idx_1d, elevation.shape)
idx

In [ ]:
elevation[0,159,131].values

In [ ]:
elevation.values[idx]

## Slicing

In [ ]:
elevation.plot()
plt.axvline(stn[0])
plt.axhline(stn[1])

In [ ]:
j,k = elevation.geometry.find_index(coords=stn)
elevation.isel(y=k[0]).plot(title="West-East")
plt.xlim(12.2, 13.2)

In [ ]:
elevation.isel(x=j[0]).plot(title="South-North")

## Writing data

### Geographical coordinates

In [ ]:
data_i = np.arange(24).reshape(2,4,3)
data_i.shape

In [ ]:
plt.imshow(data_i[0]) # first timestep
plt.colorbar()

In [ ]:
grid = mikeio.Grid2D(x0 = 12.0, dx= 1.0, y0= 55.0, dy=1.0, nx=3, ny=4, projection="LONG/LAT")
grid

In [ ]:
import pandas as pd

nt = data_i.shape[0]

da = mikeio.DataArray(data = data_i,
                      geometry=grid,
                      time = pd.date_range("2020",periods=nt,freq='D'),
                      item= mikeio.ItemInfo("Example", mikeio.EUMType.Elevation))

da

In [ ]:
ds = mikeio.Dataset([da])
ds

In [ ]:
ds.to_dfs("example_geo.dfs2")

Screenshot from MIKE Zero (viridis palette).

:::{note}
Note the location of the origin.
:::


![](images/example_dfs2_geo.png)

### Projected coordinates


Projection string in WKT format. Use MIKE Zero for formatting. Some shorthands are available for UTM zones, e.g. UTM-32.

In [ ]:
pgrid = mikeio.Grid2D(x0 = 337100.0, dx= 10000.0, y0= 6122900.0, dy=10000.0, nx=3, ny=4, projection="UTM-32")
pgrid

In [ ]:
pda = mikeio.DataArray(data = data_i,
                       geometry=pgrid,
                       time = pd.date_range("2020",periods=nt,freq='D'),
                       item= mikeio.ItemInfo("Example", mikeio.EUMType.Elevation))

pda

In [ ]:
pda.plot()

In [ ]:
pda.to_dfs("example_proj.dfs2")

Screenshot from MIKE Zero using a projected coordinate system.

![](images/example_dfs2_proj.png)